# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [8]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [9]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [10]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [11]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [12]:
df = pd.DataFrame(data['Quantity'].groupby([data['ProductName'], data['CustomerID']]).sum())
df.head(10)

Quantity
ProductName               CustomerID          
Anchovy Paste - 56 G Tube 639                1
                          883                1
                          891                1
                          2566               1
                          2902               1
                          3535               1
                          3544               1
                          5921               2
                          5986               2
                          6352               2

In [13]:
pivot = pd.pivot_table(df,index=["CustomerID"], columns=["ProductName"], values = ["Quantity"], fill_value=0)
pivot.head(10)

Quantity                                    \
ProductName Anchovy Paste - 56 G Tube Appetizer - Mini Egg Roll, Shrimp   
CustomerID                                                                
33                                  0                                 0   
200                                 0                                 0   
264                                 0                                 0   
356                                 0                                 0   
412                                 0                                 0   
464                                 0                                 0   
477                                 0                                 0   
639                                 1                                 0   
649                                 0                                 0   
669                                 0                                 0   

                                                                 \
ProductName Appetizer - Mushroom Tart Appetizer - Sausage Rolls   
CustomerID                                                        
33                                  0                         0   
200                                 0                         0   
264                                 0                         0   
356                                 0                         0   
412                                 0                         0   
464                                 0                         0   
477                                 0                         0   
639                                 1                         0   
649                                 0                         0   
669                                 0                         0   

                                                               \
ProductName Apricots - Dried Apricots - Halves Apricots Fresh   
CustomerID                                                      
33                         1                 0              0   
200                        0                 0              0   
264                        0                 1              1   
356                        0                 0              0   
412                        1                 0              0   
464                        0                 0              1   
477                        0                 0              0   
639                        0                 0              0   
649                        0                 0              0   
669                        0                 0              0   

                                                                          ...  \
ProductName Arizona - Green Tea Artichokes - Jerusalem Assorted Desserts  ...   
CustomerID                                                                ...   
33                            0                      0                 1  ...   
200                           0                      0                 0  ...   
264                           0                      0                 0  ...   
356                           0                      0                 0  ...   
412                           0                      0                 0  ...   
464                           0                      0                 0  ...   
477                           0                      0                 0  ...   
639                           0                      0                 1  ...   
649                           1                      0                 0  ...   
669                           0                      0                 0  ...   

                                                                  \
ProductName Wine - White, Colubia Cresh Wine - White, Mosel Gold   
CustomerID                                                         
33                                    0                        0   
200      

In [14]:
squareform(pdist(pivot.T, 'euclidean'))

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

In [15]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)

distances.head()

Quantity  \
ProductName                                Anchovy Paste - 56 G Tube   
         ProductName                                                   
Quantity Anchovy Paste - 56 G Tube                          1.000000   
         Appetizer - Mini Egg Roll, Shrimp                  0.003559   
         Appetizer - Mushroom Tart                          0.003927   
         Appetizer - Sausage Rolls                          0.004175   
         Apricots - Dried                                   0.004102   

                                                                              \
ProductName                                Appetizer - Mini Egg Roll, Shrimp   
         ProductName                                                           
Quantity Anchovy Paste - 56 G Tube                                  0.003559   
         Appetizer - Mini Egg Roll, Shrimp                          1.000000   
         Appetizer - Mushroom Tart                                  0.003670   
         Appetizer - Sausage Rolls                                  0.003697   
         Apricots - Dried                                           0.003692   

                                                                      \
ProductName                                Appetizer - Mushroom Tart   
         ProductName                                                   
Quantity Anchovy Paste - 56 G Tube                          0.003927   
         Appetizer - Mini Egg Roll, Shrimp                  0.003670   
         Appetizer - Mushroom Tart                          1.000000   
         Appetizer - Sausage Rolls                          0.004234   
         Apricots - Dried                                   0.003849   

                                                                      \
ProductName                                Appetizer - Sausage Rolls   
         ProductName                                                   
Quantity Anchovy Paste - 56 G Tube                          0.004175   
         Appetizer - Mini Egg Roll, Shrimp                  0.003697   
         Appetizer - Mushroom Tart                          0.004234   
         Appetizer - Sausage Rolls                          1.000000   
         Apricots - Dried                                   0.004078   

                                                                               \
ProductName                                Apricots - Dried Apricots - Halves   
         ProductName                                                            
Quantity Anchovy Paste - 56 G Tube                 0.004102          0.003616   
         Appetizer - Mini Egg Roll, Shrimp         0.003692          0.003413   
         Appetizer - Mushroom Tart                 0.003849          0.003789   
         Appetizer - Sausage Rolls                 0.004078          0.003974   
         Apricots - Dried                          1.000000          0.003755   

                                                                               \
ProductName                                Apricots Fresh Arizona - Green Tea   
         ProductName                                                            
Quantity Anchovy Paste - 56 G Tube               0.003767            0.003876   
         Appetizer - Mini Egg Roll, Shrimp       0.003479            0.003900   
         Appetizer - Mushroom Tart               0.003819            0.004120   
         Appetizer - Sausage Rolls               0.003808            0.004169   
         Apricots - Dried                        0.003722            0.004073   

                                                                   \
ProductName                                Artichokes - Jerusalem   
         ProductName                                                
Quantity Anchovy Paste - 56 G Tube                       0.004024   
         Appetizer - Mini Egg Roll, Shrimp               0.003789   
         Appetizer - Mushroom Tart           

In [16]:


distances['Quantity'].columns

distances['Quantity', 'Apricots - Dried'].head()

          ProductName                      
Quantity  Anchovy Paste - 56 G Tube            0.004102
          Appetizer - Mini Egg Roll, Shrimp    0.003692
          Appetizer - Mushroom Tart            0.003849
          Appetizer - Sausage Rolls            0.004078
          Apricots - Dried                     1.000000
Name: (Quantity, Apricots - Dried), dtype: float64

### Step 2: Get the products purchased for a specific customer of your choice.

In [17]:
customer = data[data['CustomerID']==200]
customer.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
495,200,Rene,Mercado,4820933,432,Beer - Original Organic Lager,1
663,200,Rene,Mercado,1924085,428,Wine - Ruffino Chianti,1
2213,200,Rene,Mercado,2795420,155,"Peas - Pigeon, Dry",1
2369,200,Rene,Mercado,6413024,44,"Wine - White, Colubia Cresh",1
4089,200,Rene,Mercado,2488813,53,Cassis,1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [18]:
cust_prod = customer.ProductName.tolist()
prod_similarity =[]

for product in cust_prod:
    similarities = distances['Quantity', product].sort_values(ascending=False)[1:6]
    prod_similarity.append(similarities)
#len(prod_similarity)     

In [19]:
pd.DataFrame(prod_similarity[0])
pd.DataFrame(prod_similarity[1])


Quantity
                                           Wine - Ruffino Chianti
         ProductName                                             
Quantity Raspberries - Fresh                             0.004204
         Water - Green Tea Refresher                     0.004172
         Bar - Granola Trail Mix Fruit Nut               0.004129
         Bread - Hot Dog Buns                            0.004094
         Water - Spring Water 500ml                      0.004083

In [20]:
#pd.DataFrame(prod_similarity).head(2)


In [21]:
## testing on one product and its 5 recommendations

a = pd.DataFrame(prod_similarity[0])
a.columns = a.columns.droplevel()
a = a.reset_index()
del a['level_0']
a.columns = ['prod_reco', 'similarity']
a


,prod_reco,similarity
0,Water - Spring Water 500ml,0.004164
1,Raspberries - Fresh,0.004124
2,Rabbit - Whole,0.004098
3,Cookies Cereal Nut,0.004068
4,Sauce - Rosee,0.004062


In [22]:
totalprod = []
for i in range(len(prod_similarity)):
    a = pd.DataFrame(prod_similarity[i])
    a.columns = a.columns.droplevel()
    a = a.reset_index()
    del a['level_0']
    a.columns = ['prod_reco', 'similarity']
    totalprod.append(a)

In [23]:
totalprod_reco = pd.concat(totalprod)
totalprod_reco.head(20)

,prod_reco,similarity
0,Water - Spring Water 500ml,0.004164
1,Raspberries - Fresh,0.004124
2,Rabbit - Whole,0.004098
3,Cookies Cereal Nut,0.004068
4,Sauce - Rosee,0.004062
0,Raspberries - Fresh,0.004204
1,Water - Green Tea Refresher,0.004172
2,Bar - Granola Trail Mix Fruit Nut,0.004129
3,Bread - Hot Dog Buns,0.004094
4,Water - Spring Water 500ml,0.004083


In [24]:
top5 = totalprod_reco.sort_values(by='similarity',ascending=False).head()
top5

,prod_reco,similarity
0,Rabbit - Whole,0.004828
0,Rabbit - Whole,0.004693
0,Rabbit - Whole,0.004630
1,Water - Green Tea Refresher,0.004620
2,Bread - Hot Dog Buns,0.004617


In [25]:
notpurchased = []
for product in top5['prod_reco']:
    #print(product)
    if product not in customer['ProductName']:
        notpurchased.append(product)
notpurchased

['Rabbit - Whole',
 'Rabbit - Whole',
 'Rabbit - Whole',
 'Water - Green Tea Refresher',
 'Bread - Hot Dog Buns']

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [26]:
from collections import Counter
counter = Counter(totalprod_reco['prod_reco'])

counter =dict(counter)
key=list(counter.keys())
value=list(counter.values())

import operator
sorted_x = sorted(counter.items(), key=operator.itemgetter(1), reverse=True)

In [27]:
#top 5

top5 = [k for k,v in sorted_x[:5]]
top5


['Water - Green Tea Refresher',
 'Rabbit - Whole',
 'Scallops 60/80 Iqf',
 'Bar - Granola Trail Mix Fruit Nut',
 'Bread - Hot Dog Buns']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [28]:
recommendations = {}

In [29]:
listCustomerID = data['CustomerID'].unique().tolist()

In [30]:
for id_ in listCustomerID:
    customer = data[data['CustomerID']==id_]
    cust_prod = customer.ProductName.tolist()

    prod_similarity =[]

    for product in cust_prod:
        similarities = distances['Quantity', product].sort_values(ascending=False)[1:6]
        prod_similarity.append(similarities)

    totalprod = []

    for i in range(len(prod_similarity)):
        a = pd.DataFrame(prod_similarity[i])
        a.columns = a.columns.droplevel()
        a = a.reset_index()
        del a['level_0']
        a.columns = ['prod_reco', 'similarity']
        totalprod.append(a)
    totalprod_reco = pd.concat(totalprod)
    
    counter = Counter(totalprod_reco['prod_reco'])
    counter =dict(counter)
    sorted_x = sorted(counter.items(), key=operator.itemgetter(1), reverse=True)
 
    recommendations[id_] = [k for k,v in sorted_x[:5]]

In [ ]:
recommendations

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
result = pd.DataFrame(recommendations)
result2 = result.T
result2.columns = ['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
result2['CustomerID'] = result2.index
new_order = [5, 0, 1, 2, 3, 4]
result2 = result2[result2.columns[new_order]]
result2.head()

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [254]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [260]:
pd.DataFrame({'new_customer': new_customer})

,new_customer
Bandage - Flexible Neon,3
Beef - Montreal Smoked Brisket,4
Cookies - Assorted,3
Fenngreek Seed,1
Flavouring - Orange,3
French Pastry - Mini Chocolate,4
Oil - Shortening - All - Purpose,2
"Pepper - White, Ground",2
Snapple Lemon Tea,5
"Sole - Dover, Whole, Fresh",4


In [258]:
pivot = pd.pivot_table(df,index=["ProductName"], columns=["CustomerID"], values = ["Quantity"], fill_value=0)
pivot.head(10)
#distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
#                         index=data.columns, columns=data.columns)

#similarities = distances['new_customer'].sort_values(ascending=False)[1:]
#similarities


Quantity                                  \
CustomerID                              33 200 264 356 412 464 477 639 649   
ProductName                                                                  
Anchovy Paste - 56 G Tube                0   0   0   0   0   0   0   1   0   
Appetizer - Mini Egg Roll, Shrimp        0   0   0   0   0   0   0   0   0   
Appetizer - Mushroom Tart                0   0   0   0   0   0   0   1   0   
Appetizer - Sausage Rolls                0   0   0   0   0   0   0   0   0   
Apricots - Dried                         1   0   0   0   1   0   0   0   0   
Apricots - Halves                        0   0   1   0   0   0   0   0   0   
Apricots Fresh                           0   0   1   0   0   1   0   0   0   
Arizona - Green Tea                      0   0   0   0   0   0   0   0   1   
Artichokes - Jerusalem                   0   0   0   0   0   0   0   0   0   
Assorted Desserts                        1   0   0   0   0   0   0   1   0   

                                       ...                                \
CustomerID                        669  ... 97753 97769 97793 97900 97928   
ProductName                            ...                                 
Anchovy Paste - 56 G Tube           0  ...    25     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp   0  ...    25     0     0     0     0   
Appetizer - Mushroom Tart           0  ...     0     0     0     0     0   
Appetizer - Sausage Rolls           0  ...     0     0     0    25    25   
Apricots - Dried                    0  ...    25     0     0     0     0   
Apricots - Halves                   0  ...     0     0     0    25    50   
Apricots Fresh                      0  ...     0     0     0     0     0   
Arizona - Green Tea                 0  ...     0     0     0     0    25   
Artichokes - Jerusalem              0  ...     0     0     0     0     0   
Assorted Desserts                   0  ...     0     0     0     0     0   

                                                          new_customer  
CustomerID                        98069 98159 98185 98200               
ProductName                                                             
Anchovy Paste - 56 G Tube             0     0     0     0          NaN  
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0          NaN  
Appetizer - Mushroom Tart             0     0    25     0          NaN  
Appetizer - Sausage Rolls            25     0    25     0          NaN  
Apricots - Dried                      0     0     0     0          NaN  
Apricots - Halves                    25     0    25    25          NaN  
Apricots Fresh                        0     0     0    25          NaN  
Arizona - Green Tea                   0     0     0     0          NaN  
Artichokes - Jerusalem                0     0     0     0          NaN  
Assorted Desserts                    25     0     0    25          NaN  

[10 rows x 1001 columns]

### Step 8: Recommend 5 products to this new customer using a item similarity approach.